In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("htttp://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted.")
else:
    print(f"Index '{index_name}' does not exist.")

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.autonotebook import tqdm 

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Source: {hit['_source']}")

In [ ]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Question: {hit['_source']['question']}, Text: {hit['_source']['text']}\n")

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
        
    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""

    for doc in search_results:
        context_entry = context_template.format(question=doc['question'], text=doc['text'])
        context += f"{context_entry}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

prompt = build_prompt(query, result_docs)
len(prompt)

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

tokens = encoding.encode(prompt)

num_tokens = len(tokens)
num_tokens

In [ ]:
encoding.decode_single_token_bytes(63842)